# Handwritten Text Recognition - OCR - CNN

In [80]:
# Imports
import cv2
import numpy as np
import os
import random
import matplotlib.pyplot as plt

In [81]:
# Data Acquisition
DATA = "../data/raw"
LETS = list("SINO")

In [82]:

def load_letters(path, letters_list):
    letters = []
    # Looping through each letter
    for let in letters_list:
        images = os.listdir(os.path.join(path, let))
        # Looping through each letter image in its corresponding folder
        for pic in images:
            # Reading letter image, converting it into a gray scale and resizing it
            img = cv2.imread(os.path.join(path, let, pic))
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (100, 100))
            
            # Saving letter image array and target into a list
            letters.append([resized, let])
    
    return letters
        
letters = load_letters(DATA, LETS)
print(len(letters))

72


In [ ]:
def check_loaded_data(letters_list):
    y = [letters[1] for target in letters]

In [114]:
# Checking loaded images
def shuffle(list_to_shuffle):
    random.shuffle(list_to_shuffle)
    return list_to_shuffle

shuffled = shuffle(letters)
print(len(shuffled))

72
